Introduction
---

RKnot's purpose is two-fold:

1) Investigate the Herd Immunity Threshold (HIT) under Conditions of Heterogeneity
2) Investigate impacts of policy decisions on viral spread, hospital capacity, and death

Standard $R_0$ models estimate HIT as:

$HIT = 1 - 1/R_0$ [(ref)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3782273/)
           
For a virus with $R_0$ of 2.5 ([as is estimated for sars-cov-2](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html)), this results in a HIT of 60%. 
Yet, measurements of seroprevalance of antibodies have suggested 15-20% peaks in some communities ([e.g. New York City](https://www.medrxiv.org/content/10.1101/2020.06.28.20142190v1)). *Is it possible that HIT is lower than* $R_0$ *would suggest?*

$R_0$ carries several basic assumptions that are not applicable to real world populations. The key assumption is homogeneity, meaning specifically:

1. an infected subject exists in a pool otherwise entirely constituted of susceptible subjects 
2. an infected subject has the same number of contacts with susceptible subjects as any other infected subject
3. each contact is equally likely to result in an infection, regardless of where, when, or between whom the contact was made

#### Heterogeneity 
RKnot seeks to address the shortfalls in $R_0$ models by allowing the user to introduce customized, heterogeneous populations. Populations can have heterogeneity across several axis including:

+ Fatality Rate
+ Population Density
+ Movement - frequency and distance of movements in the space
+ Susceptibility - segments of population can be made immune (without requiring vaccination) to mimick phenomenon like [possible T-cell immunity](https://www.bmj.com/content/370/bmj.m3563).
+ $T$ or Transmission Factor: RKnot (as with $R_0$) assumes that all contacts have the same likelihood of transmission, $\tau$. This is a homogenous assumption, however, RKnot introduces a unitless Transmission Factor, $T$, for each subject at each contact that can modulate $\tau$. This could mimick social distancing or mask wearing or different socio-cultural norms that may impact spread (i.e. east Asian bows versus southern European double-kisses)

Future Additions:
+ Infectiousness Curve - $R_0$ models assume an infected subject is equally likely to transmit at any point during the infection, however, [research suggests](https://www.ijidonline.com/article/S1201-9712%2820%2930119-3/pdf) viral load and, thereby, likelihood to transmit follows a distribution curve. RKnot does *not currently support this feature*, however, it is being added.
+ Isolation - with the heightened awareness of a pandemic, individuals in population are more likely to self-isolate or quarantine themselves, thereby helping to reduce spread.

Where RKnot seems to distinguish itself from prevailing models is that to affect the heterogeneity above, RKnot simulates *every contact of every subject in the population*.

In order to do that, RKnot must determine the $ \tau $ for the space created by the user.

#### Approach

${R_0}$ is typically assessed at the population level and individual level ([and often those
assessments disagree](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1804098/)). The basic SIR model evaluates ${R_0}$ as:

$R_0=\beta*𝑑$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;(see [R0 in Simple Models](https://web.stanford.edu/~jhj1/teachingdocs/Jones-on-R0.pdf))

β = transmission rate, or number of secondary of infections per unit time (typically days)

𝑑 = infectious period, or infectiousness duration per unit time (typically days)

This is often rewritten as:

$R_0=\beta/γ$

where:

γ = recovery rate, or the expected recovery of infections per unit time (typically days)

So, $R_0$ can be read as the fraction of new infections vs. recovering infections over the 
"life" of an infection.

Thus, modeling on a per contact or per subject basis is hampered b/c Ro models often 
average out contacts and focus more broadly on number of transmissions during 
the infection period (and assume contacts average out among the population).

To model each contact, we must know the probability of transmission for each contact, $\tau$. The transmission rate $\beta$ does imply a likelihood of infection, if the number of contacts is known and can be written as:

$\beta=\tau*c$

where   𝞽 = likelihood of infection during a contact
        c = number of contacts / unit time,  (i.e. contact rate)

Thus,

$R_0=\tau*c*𝑑$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;([see equation 1](https://web.stanford.edu/~jhj1/teachingdocs/Jones-on-R0.pdf))

or 

$R_0=\tau*c*1/\gamma$

For simulation purposes, we assume:

+ $R_0$ is known (from external analysis and provided by the user)
+ $\gamma$ is known (from external analysis and provided by the user)

Thus, unknowns are    $\tau$ & c

The simulation space, however, is given a number of parameters that allow us to estimate
the expected number of contacts. We know:

+ The population size
+ The number of locations
+ The movement patterns of subjects
+ The likelihood that a subject will be at a particular location at one time given 1/2/3 above
        

#### Expected Contacts 

$R_0$ assumes that an infected subject will hvae as many contacts with the population as any other subject. A simple method to estimate contacts for one infected subject is to, then, assume that each subject is equally likely to be in any one location at any time (in Rknot, this implemented by the `equal` mover function).

The probability of a single dot being in a singe location is:

$P(LOC_{xy},DOT_i)=1/N$ 

   where  $_{xy}$ = x/y coordinates of the location 
          $N$ = number of locations
          
The probability of another dot being there at the same time:

$P(LOC_{xy},DOT_{ij})=1/N*1/N$

The probability of $n$ dots being at the same location at the same time:

$P(LOC_{xy},DOT_{ij})=1/N^n$

Then, the number of ordered contacts is:

$\sum_{n=1}^N(1 / N^n)$ 

And for all possible orders:

$\sum_{n=1}^N(1/N^n)*n$ 

#### Likelihood of Transmission

With expected contact rate known, probability of transmission is found as:

$\tau=R_0/(𝑑*c)$

or

$\tau=R_0*γ/c$  where $0<=\tau<=1$
        
        
Checking dimenions,

$=\frac{infections}{1}*\frac{1}{days}*\frac{days}{contacts}$
$=\frac{infections}{contacts}$
            
This is the likelihood of transmission of a contact and is a fundamental factor used in the RKnot simulation. 

If a susceptible subject contacts an infected, anywhere in the space, at any time,
its probability of infection is $\tau$.

If a susceptible comes in contact with multiple infected at any time, its probability of
infection is    $\tau*n_{inf}$

#### The Average Contact
RKnot assumes that each and every contact is an "Average Contact". The Average Contact is a purely theoretical interaction that would result in about an average likelihood of transmission relative to all other possible interactions. The Average Contact is uninfluenced by external factors such as the demographics of the subjects, etc. Thus, the likelihood of transmission of an average contact is a fundamental property of the virus in question. And we would expect that more virulent viruses would have higher likelihood of transmission in an Average Contact, or that viruses with a higher $R_0$ would have a higher likelihood of transmission in an Average Contact.

I like to think of the Average Contact as two people on an elevator, standing two feet apart, having a conversation. This to me feels about average, or in the middle, of other contacts like, say, two people standing in a open field, 6 feet apart, having a conversation, or two people pressed right up against each other, by many other people, on the floor section of a concert.

***
#### References
+ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3782273/
+ https://www.bmj.com/content/370/bmj.m3563
+ https://www.medrxiv.org/content/10.1101/2020.06.28.20142190v1
+ https://fivethirtyeight.com/features/without-a-vaccine-herd-immunity-wont-save-us/
+ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3935673/
+ https://web.stanford.edu/~jhj1/teachingdocs/Jones-on-R0.pdf
+ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1804098/
+ https://www.ijidonline.com/article/S1201-9712%2820%2930119-3/pdf